# Preparing data

To run the models, we need the following input: 
- X_train_res and y_train_res: resampled train data using SMOTE
- X_test and y_test 
- X_train_12: with only the measurements for stage 1 and 2 
- X_test_12: with only the measurements for stage 1 and 2 

Import libraries. 

In [54]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix 
from collections import Counter

Standardized_complete_dataset is the dataframe I created with all the extracted features from TS fresh. 
It has 111 rows and 129 columns. 

In [55]:
merged_df = pd.read_csv('/Users/dionnespaltman/Desktop/V6/merged_df_04-06-2024.csv', sep=',')
merged_df.drop('Unnamed: 0.1', axis=1, inplace=True)
merged_df.drop('Unnamed: 0', axis=1, inplace=True)

display(merged_df)

,ID,Stage,Gender,Date,Location,Condition,Faintness,Dizziness,Weakness,Lightheadedness,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,23,1.0,2,2020-08-01,1,2,1.0,2.0,2.0,1.0,...,-0.336826,0.440930,-0.608761,-0.600147,-0.356101,0.239263,0.357168,0.546947,0.644834,0.452026
1,24,1.0,2,2020-01-22,1,2,1.0,1.0,1.0,1.0,...,1.774942,0.440930,2.329367,2.326860,0.188255,-0.249122,-0.025248,0.619122,-0.622877,-0.261674
2,25,1.0,2,2020-05-02,1,2,1.0,1.0,1.0,1.0,...,0.522235,0.440930,0.181549,0.373204,-0.399350,0.048667,0.213801,-0.150745,0.132589,0.169110
3,26,1.0,1,2020-06-02,2,1,2.0,2.0,2.0,1.0,...,-0.443079,0.440930,-0.888021,-0.812001,-0.898647,-0.313749,-0.079950,0.538928,-1.056506,-0.422165
4,27,1.0,1,2020-06-02,2,2,1.0,1.0,1.0,1.0,...,1.662959,1.885134,-2.304025,0.532326,2.472222,1.099701,0.935654,0.651200,3.744591,2.026626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,142,1.0,1,2021-05-31,1,3,1.0,1.0,1.0,1.0,...,-0.908121,0.440930,-1.248127,-1.305586,-0.086453,-0.578589,-0.316920,-0.559738,-0.485024,-0.490340
107,143,1.0,1,2021-01-06,1,3,1.0,1.0,1.0,1.0,...,0.879396,0.440930,0.519120,0.782538,-0.858283,-1.485397,-1.415721,-1.137138,-1.944625,-1.908812
108,144,1.0,1,2021-01-06,1,3,1.0,1.0,1.0,1.0,...,-0.617098,0.440930,-0.891987,-0.930731,-0.748169,-0.803177,-0.537660,-0.359251,-0.925818,-0.818200
109,145,1.0,2,2021-02-06,0,1,1.0,1.0,1.0,1.0,...,-0.535881,0.346928,-0.581134,-0.713336,1.189029,0.042223,0.208832,0.065780,0.032361,0.135367


In [56]:
print(list(merged_df.columns))

['ID', 'Stage', 'Gender', 'Date', 'Location', 'Condition', 'Faintness', 'Dizziness', 'Weakness', 'Lightheadedness', 'Fear', 'Tension', 'Stress', 'Physical_sum', 'Psychological_sum', 'Sum_VVR', 'Sum_12', 'Sum_456', 'Sum_4567', 'Sum_1', 'Sum_2', 'VVR_group', 'AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 'AU01_r__mean', 'AU01_r__root_mean_square', 'AU02_r__sum_values', 'AU02_r__variance', 'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__mean', 'AU02_r__root_mean_square', 'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 'AU04_r__mean', 'AU04_r__root_mean_square', 'AU05_r__sum_values', 'AU05_r__variance', 'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__mean', 'AU05_r__root_mean_square', 'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 'AU06_r__mean', 'AU06_r__root_mean_square', 'AU07_r__sum_values', 'AU07_r__variance', 'AU07_r__standard_deviatio

In [ ]:
# columns_to_drop = ['ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition', 'Gender', 'Date', 'Stage', 'Location',] 
columns_to_drop = ['Stage', 'Gender', 'Date', 'Location', 'Condition', 'Faintness',
                    'Dizziness', 'Weakness', 'Lightheadedness', 'Fear', 'Tension', 'Stress', 'Physical_sum', 
                    'Psychological_sum', 'Sum_VVR', 'Sum_12', 'Sum_456', 'Sum_4567']

merged_df = merged_df.drop(columns_to_drop, axis=1)

In [ ]:
print(list(merged_df.columns))

These are the columns I use to predict, so all my features. I need these as a list to establish my featurizer. 
I have 102 features from TS fresh and then I added the two VVR measurements from stage 1 and 2. 

In [32]:
columns_ts_fresh = ['AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 
                    'AU01_r__mean', 'AU01_r__root_mean_square', 'AU02_r__sum_values', 'AU02_r__variance', 
                    'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__mean', 'AU02_r__root_mean_square', 
                    'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 
                    'AU04_r__mean', 'AU04_r__root_mean_square', 'AU05_r__sum_values', 'AU05_r__variance',
                    'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__mean', 'AU05_r__root_mean_square', 
                    'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 
                    'AU06_r__mean', 'AU06_r__root_mean_square', 'AU07_r__sum_values', 'AU07_r__variance', 
                    'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__mean', 'AU07_r__root_mean_square', 
                    'AU09_r__sum_values', 'AU09_r__variance', 'AU09_r__standard_deviation', 'AU09_r__maximum',
                    'AU09_r__mean', 'AU09_r__root_mean_square', 'AU10_r__sum_values', 'AU10_r__variance', 
                    'AU10_r__standard_deviation', 'AU10_r__maximum', 'AU10_r__mean', 'AU10_r__root_mean_square',
                    'AU12_r__sum_values', 'AU12_r__variance', 'AU12_r__standard_deviation', 'AU12_r__maximum', 
                    'AU12_r__mean', 'AU12_r__root_mean_square', 'AU14_r__sum_values', 'AU14_r__variance', 
                    'AU14_r__standard_deviation', 'AU14_r__maximum', 'AU14_r__mean', 'AU14_r__root_mean_square', 
                    'AU15_r__sum_values', 'AU15_r__variance', 'AU15_r__standard_deviation', 'AU15_r__maximum', 
                    'AU15_r__mean', 'AU15_r__root_mean_square', 'AU17_r__sum_values', 'AU17_r__variance', 
                    'AU17_r__standard_deviation', 'AU17_r__maximum', 'AU17_r__mean', 'AU17_r__root_mean_square', 
                    'AU20_r__sum_values', 'AU20_r__variance', 'AU20_r__standard_deviation', 'AU20_r__maximum', 
                    'AU20_r__mean', 'AU20_r__root_mean_square', 'AU23_r__sum_values', 'AU23_r__variance', 
                    'AU23_r__standard_deviation', 'AU23_r__maximum', 'AU23_r__mean', 'AU23_r__root_mean_square', 
                    'AU25_r__sum_values', 'AU25_r__variance', 'AU25_r__standard_deviation', 'AU25_r__maximum', 
                    'AU25_r__mean', 'AU25_r__root_mean_square', 'AU26_r__sum_values', 'AU26_r__variance', 
                    'AU26_r__standard_deviation', 'AU26_r__maximum', 'AU26_r__mean', 'AU26_r__root_mean_square', 
                    'AU45_r__sum_values', 'AU45_r__variance', 'AU45_r__standard_deviation', 'AU45_r__maximum', 
                    'AU45_r__mean', 'AU45_r__root_mean_square']

print(len(columns_ts_fresh))

102


# Train and test split

OLD: 
In total, we have 104 participants.

I started with a test size of 20%. Then there are 83 people in the train set and 21 in the test set. 
With a test size of 30#, there are 72 people in the train set and 32 in the test set. 

Naturally, we stratify on VVR_group. 

NEW: 
In total, we have 111 participants.

I started with a test size of 20%. Then there are 88 people in the train set and 23 in the test set. 
With a test size of 30% there are 77 people in the train set and 34 in the test set. 

Naturally, we stratify on VVR_group. 

In [63]:
train, test = train_test_split(merged_df, test_size=0.3, random_state=123, stratify=merged_df['VVR_group'])

print(train.shape)
print(test.shape)

(77, 106)
(34, 106)


OLD: Unfortunately, the test set is very small with only 7 people in the high VVR condition. 

NEW: The test set is very small with only 11 people in the high VVR condition. 

In [65]:
columns_to_drop = [ 'ID', 'VVR_group'] 

X_test = test.drop(columns_to_drop, axis=1)
y_test = test['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_test))

Original dataset shape Counter({0: 23, 1: 11})


# X_test plus gender 

In [68]:
columns_to_drop = [ 'ID'] 
X_test_gender = test.drop(columns_to_drop, axis=1)
display(X_test_gender.head())

,Sum_1,Sum_2,VVR_group,AU01_r__sum_values,AU01_r__variance,AU01_r__standard_deviation,AU01_r__maximum,AU01_r__mean,AU01_r__root_mean_square,AU02_r__sum_values,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
80,9.0,11.0,0,-1.954248,-0.951158,-0.794258,-2.000083,0.255940,-0.532746,-1.756362,...,-0.166497,-0.729816,2.825468,1.673580,-2.233545,0.289550,0.393906,-0.511621,1.873165,0.899641
62,11.0,10.0,0,-0.531321,-0.178666,0.044588,0.504332,-0.398439,-0.134585,-0.306588,...,-0.363370,-0.080351,-0.509222,-0.565219,-0.579429,-0.022861,0.158174,0.466753,-0.568289,-0.075143
67,11.0,10.0,0,-0.293895,0.814623,0.815957,0.504332,0.187131,0.677928,0.120579,...,0.567536,0.440930,0.271424,0.449194,-0.116745,0.791976,0.739793,0.129936,0.675229,0.802541
43,8.0,8.0,0,-0.462186,-0.190678,0.033761,0.504332,-0.500519,-0.171970,0.077514,...,0.187859,0.346928,-0.008672,0.055083,-0.336010,-0.518784,-0.261437,-0.190843,-0.460154,-0.433136
105,8.0,8.0,0,1.797969,2.225527,1.667268,0.504332,1.289613,1.707848,1.420513,...,-0.077072,0.440930,-0.477280,-0.360369,0.855433,0.216467,0.340371,0.202111,0.358639,0.349728


In [69]:
X_test_gender.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_test_gender.csv')

# Applying SMOTE

In [70]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

We apply SMOTE on the train data. The strategy is to make the minority as big as the majority class. 

In [73]:
columns_to_drop = [ 'ID', 'VVR_group'] 

X_train = train.drop(columns_to_drop, axis=1)
y_train = train['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_train))

# Apply SMOTE to the training data with sampling strategy set to 'auto' (default)
sm = SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=5)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Print resampled class distribution
print('Resampled dataset shape %s' % Counter(y_train_res))

Original dataset shape Counter({0: 51, 1: 26})
Resampled dataset shape Counter({1: 51, 0: 51})


In [75]:
display(X_train_res)
X_train_res.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_train_resampled_gender.csv')

,Sum_1,Sum_2,AU01_r__sum_values,AU01_r__variance,AU01_r__standard_deviation,AU01_r__maximum,AU01_r__mean,AU01_r__root_mean_square,AU02_r__sum_values,AU02_r__variance,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,12.000000,13.000000,-0.114682,-0.412734,-0.175865,0.415941,-0.687555,-0.400360,0.970556,0.364929,...,-0.588472,0.201653,-1.080948,-1.005558,-1.057202,-1.532650,-1.499128,-1.273469,-2.129886,-2.037173
1,17.000000,16.000000,-0.038570,-0.768864,-0.561401,0.504332,-0.809168,-0.758472,0.333551,-0.368145,...,0.289276,0.440930,0.189637,0.225341,1.063062,0.406128,0.477464,0.619122,-0.068091,0.354797
2,16.000000,12.000000,-0.436004,0.206480,0.369342,0.158133,-0.021607,0.244861,-0.836108,-0.490845,...,0.522235,0.440930,0.181549,0.373204,-0.399350,0.048667,0.213801,-0.150745,0.132589,0.169110
3,8.000000,8.000000,0.558569,0.560424,0.637285,0.504332,0.287352,0.558101,0.411212,0.165073,...,-0.045264,0.440930,-0.462771,-0.331676,-0.090762,-0.747721,-0.481159,0.330422,-0.600511,-0.671622
4,9.000000,8.000000,0.039454,0.762845,0.780378,0.364379,0.528676,0.748981,-0.131359,0.091414,...,-0.535630,0.440930,-0.856850,-0.858808,0.063929,0.829485,0.764250,0.619122,0.779664,0.855063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,10.258780,9.517560,0.385373,-0.136089,0.063036,0.504332,-0.085024,-0.025875,0.648297,0.390875,...,0.087061,0.440930,-0.303493,-0.162647,0.935821,1.277282,1.026884,0.571391,0.484029,1.017265
98,10.325045,11.000000,0.530122,0.128923,0.269900,0.504332,0.064515,0.189538,0.397173,-0.001319,...,0.463430,0.440930,0.163846,0.329037,0.728172,0.306648,0.406269,0.413024,0.300445,0.392270
99,15.129734,13.064867,-0.349330,-0.767647,-0.564240,0.188751,-0.993996,-0.810230,0.240059,-0.428729,...,-0.661940,-0.229599,-1.138173,-1.083741,1.441259,0.718178,0.679021,0.243783,0.649599,0.740157
100,18.399660,16.839456,1.025860,1.212310,1.008786,0.370858,1.041588,1.098296,0.085972,-0.148117,...,0.914189,0.223561,1.081991,1.105477,1.181302,1.038896,0.862400,0.449104,1.422642,1.139505


# Adding class weights

I will add class weights to my models, because of my inbalanced data set. 
https://medium.com/@ravi.abhinav4/improving-class-imbalance-with-class-weights-in-machine-learning-af072fdd4aa4 

Results: 
{0: 0.6428571428571429, 1: 2.25}

NEW: Class weights: {0: 0.7549019607843137, 1: 1.4807692307692308}

In [76]:
import numpy as np

def calculate_class_weights(y):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    total_samples = len(y)
    class_weights = {}

    for class_label, class_count in zip(unique_classes, class_counts):
        class_weight = total_samples / (2.0 * class_count)
        class_weights[class_label] = class_weight

    return class_weights

# Assuming 'y' contains the class labels (0s and 1s) for the binary classification problem
class_weights = calculate_class_weights(y_train)
print("Class weights:", class_weights)


Class weights: {0: 0.7549019607843137, 1: 1.4807692307692308}


# Save X_train_res, y_train_res, X_test, y_test

In [77]:
display(X_train_res)

,Sum_1,Sum_2,AU01_r__sum_values,AU01_r__variance,AU01_r__standard_deviation,AU01_r__maximum,AU01_r__mean,AU01_r__root_mean_square,AU02_r__sum_values,AU02_r__variance,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,12.000000,13.000000,-0.114682,-0.412734,-0.175865,0.415941,-0.687555,-0.400360,0.970556,0.364929,...,-0.588472,0.201653,-1.080948,-1.005558,-1.057202,-1.532650,-1.499128,-1.273469,-2.129886,-2.037173
1,17.000000,16.000000,-0.038570,-0.768864,-0.561401,0.504332,-0.809168,-0.758472,0.333551,-0.368145,...,0.289276,0.440930,0.189637,0.225341,1.063062,0.406128,0.477464,0.619122,-0.068091,0.354797
2,16.000000,12.000000,-0.436004,0.206480,0.369342,0.158133,-0.021607,0.244861,-0.836108,-0.490845,...,0.522235,0.440930,0.181549,0.373204,-0.399350,0.048667,0.213801,-0.150745,0.132589,0.169110
3,8.000000,8.000000,0.558569,0.560424,0.637285,0.504332,0.287352,0.558101,0.411212,0.165073,...,-0.045264,0.440930,-0.462771,-0.331676,-0.090762,-0.747721,-0.481159,0.330422,-0.600511,-0.671622
4,9.000000,8.000000,0.039454,0.762845,0.780378,0.364379,0.528676,0.748981,-0.131359,0.091414,...,-0.535630,0.440930,-0.856850,-0.858808,0.063929,0.829485,0.764250,0.619122,0.779664,0.855063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,10.258780,9.517560,0.385373,-0.136089,0.063036,0.504332,-0.085024,-0.025875,0.648297,0.390875,...,0.087061,0.440930,-0.303493,-0.162647,0.935821,1.277282,1.026884,0.571391,0.484029,1.017265
98,10.325045,11.000000,0.530122,0.128923,0.269900,0.504332,0.064515,0.189538,0.397173,-0.001319,...,0.463430,0.440930,0.163846,0.329037,0.728172,0.306648,0.406269,0.413024,0.300445,0.392270
99,15.129734,13.064867,-0.349330,-0.767647,-0.564240,0.188751,-0.993996,-0.810230,0.240059,-0.428729,...,-0.661940,-0.229599,-1.138173,-1.083741,1.441259,0.718178,0.679021,0.243783,0.649599,0.740157
100,18.399660,16.839456,1.025860,1.212310,1.008786,0.370858,1.041588,1.098296,0.085972,-0.148117,...,0.914189,0.223561,1.081991,1.105477,1.181302,1.038896,0.862400,0.449104,1.422642,1.139505


In [78]:
X_train_res.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_train_resampled.csv')
y_train_res.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_train_resampled.csv')
X_test.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_test.csv')
y_test.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_test.csv')

# X_train_12 and X_test_12 

In this set we want to drop all the action units. 

In [80]:
columns = columns_ts_fresh

print(X_train.shape)
print(X_test.shape)

X_train_12 = X_train.drop(columns=columns)
X_test_12 = X_test.drop(columns=columns)

# columns_to_drop = ['Stage', 'Gender', 'Location', 'Condition', 'Faintness', 'Dizziness',
#                  'Weakness', 'Lightheadedness', 'Fear', 'Tension', 'Stress',
#                 'Physical_sum', 'Psychological_sum', 'Sum_VVR']

# columns_to_drop_test = ['Stage', 'Location', 'Faintness', 'Dizziness', 'Weakness',
#                     'Lightheadedness', 'Fear', 'Tension', 'Stress', 'Physical_sum',
#                     'Psychological_sum', 'Sum_VVR']

# X_train_12 = X_train_12.drop(columns=columns_to_drop, axis=1)
# X_test_12 = X_test_12.drop(columns=columns_to_drop_test, axis=1)

print(X_train_12.shape)
print(X_test_12.shape)

(77, 104)
(34, 104)
(77, 2)
(34, 2)


In [81]:
display(X_train_12)
display(X_test_12)

,Sum_1,Sum_2
89,12.0,13.0
20,17.0,16.0
2,16.0,12.0
70,8.0,8.0
15,9.0,8.0
...,...,...
72,9.0,9.0
38,10.0,10.0
29,10.0,9.0
101,10.0,9.0


,Sum_1,Sum_2
80,9.0,11.0
62,11.0,10.0
67,11.0,10.0
43,8.0,8.0
105,8.0,8.0
88,10.0,10.0
24,8.0,8.0
5,10.0,9.0
79,10.0,8.0
100,11.0,8.0


In [82]:
X_train_12.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_train_12.csv')
X_test_12.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_test_12.csv')
y_train.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_train_12.csv')